In [39]:
import pandas as pd
import sqlite3

In [40]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'
sql = f"SELECT r.name_dish, r.manual, rc.cat_id  from recipe r " \
      f"JOIN recipe_categories rc " \
      f"ON r.id = rc.recipe_id"

loaded_data = pd.read_sql(sql, conn)
#pandas data frame
conn.close()

In [49]:
loaded_data.head(100)

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(loaded_data.manual, loaded_data.cat_id, random_state=0)

In [43]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer().fit(x_train)
normalized_X = scaler.transform(x_train)

print('До нормализации: ')
print(x_train[:3])
print()
print('После нормализации: ')
normalized_X[:3]